In [1]:
import pandas as pd
import numpy as np
import sklearn
import torch.nn as nn
import sklearn.model_selection
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
import pickle
from sklearn.metrics import accuracy_score,f1_score
pd.set_option('display.max_columns', None)

In [2]:
pwd

'/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/predictions/ml_notebooks'

In [33]:
df = pd.read_csv("/Users/liqingyang/Downloads/calibrated_dataset_19.97%.csv", index_col=0)
# use list comphrension to call games from 3/18
test_set = df[5346:]
test_set = test_set.reset_index(drop=True)

# Predictors chosen using sequential feature selector trained on ridge classifier
save_path = '/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/ml_notebooks/factors/'
file_name = 'calibrated_predictors_ridge_classifier_30_predictors_19.97%_2019_2024.txt'
file_path = save_path + file_name
with open(file_path, 'r') as f:
    predictors = f.read()
    
# model
save_path = '/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/ml_notebooks/weights/'
file_name = 'calibrated_ridge_classifier_30_predictors_19.97%_2019_2024.pkl'
file_path = save_path + file_name
with open(file_path, 'rb') as f:
    clf = pickle.load(f)

predictors = predictors.split(',')[:-1]
X = test_set[predictors]
predictions = clf.predict(X)
result = pd.Series(predictions, index = X.index)
final = pd.concat([test_set[['Teams_x', 'team_next_opp_x', 'Fanduel_odds_x',
 'Draftkings_odds_x',
 'BetMGM_odds_x',
 'Fanduel_odds_y',
 'Draftkings_odds_y',
 'BetMGM_odds_y', 'date_next']], result], axis=1)
final.columns = ['Teams_x', 'Teams_y', 'Fanduel_odds_x',
 'Draftkings_odds_x',
 'BetMGM_odds_x',
 'Fanduel_odds_y',
 'Draftkings_odds_y',
 'BetMGM_odds_y', "Date", 'Predictions']
final['Actual'] = test_set['target_x']

accuracy = accuracy_score(final['Actual'], final['Predictions'])
accuracy

0.6975308641975309

In [34]:
final['max_odds']  = final[['Fanduel_odds_x', 'Draftkings_odds_x', 'BetMGM_odds_x']].max(axis=1)

In [38]:
final[final['Date'] == '2023-11-30']

,Teams_x,Teams_y,Fanduel_odds_x,Draftkings_odds_x,BetMGM_odds_x,Fanduel_odds_y,Draftkings_odds_y,BetMGM_odds_y,Date,Predictions,Actual,max_odds
308,SAS,ATL,3.45,3.50,3.40,1.33,1.32,1.33,2023-11-30,0,0,3.50
324,IND,MIA,2.08,2.10,2.10,1.79,1.77,1.74,2023-11-30,0,0,2.10
325,POR,CLE,5.30,5.25,5.28,1.17,1.17,1.17,2023-11-30,0,1,5.30
328,ATL,SAS,1.33,1.32,1.33,3.45,3.50,3.40,2023-11-30,1,1,1.33
329,CLE,POR,1.17,1.17,1.17,5.30,5.25,5.28,2023-11-30,1,0,1.17
330,GSW,LAC,1.49,1.49,1.48,2.72,2.70,2.70,2023-11-30,1,1,1.49
332,CHI,MIL,3.90,3.80,3.80,1.28,1.28,1.29,2023-11-30,0,1,3.90
333,MIN,UTA,1.18,1.19,1.18,5.20,5.00,5.00,2023-11-30,1,1,1.19
334,OKC,LAL,1.42,1.44,1.44,2.98,2.85,2.85,2023-11-30,1,1,1.44
336,BRK,CHO,1.28,1.29,1.29,3.90,3.75,3.80,2023-11-30,1,0,1.29


In [39]:
probabilities = clf.predict_proba(X)[:, 1]
final['probability'] = probabilities
final

,Teams_x,Teams_y,Fanduel_odds_x,Draftkings_odds_x,BetMGM_odds_x,Fanduel_odds_y,Draftkings_odds_y,BetMGM_odds_y,Date,Predictions,Actual,max_odds,probability
0,CHO,PHI,4.85,4.90,4.75,1.20,1.20,1.20,2023-03-17,0,0,4.90,0.180000
1,BRK,SAC,1.72,1.74,1.71,2.18,2.15,2.15,2023-03-16,1,0,1.74,0.595238
2,OKC,TOR,3.05,3.15,3.20,1.40,1.39,1.36,2023-03-16,0,0,3.20,0.264901
3,POR,BOS,2.60,2.60,2.55,1.53,1.53,1.53,2023-03-17,0,0,2.60,0.395455
4,CLE,WAS,1.48,1.48,1.44,2.76,2.80,2.80,2023-03-17,1,1,1.48,0.620690
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1777,MIA,CLE,2.10,2.10,2.05,1.77,1.77,1.77,2024-03-20,0,1,2.10,0.426471
1778,UTA,OKC,9.10,9.00,9.00,1.07,1.07,1.07,2024-03-20,0,0,9.10,0.000000
1779,POR,LAC,6.60,6.25,6.50,1.12,1.13,1.12,2024-03-20,0,0,6.60,0.180000
1780,SAC,TOR,1.16,1.16,1.15,5.60,5.55,5.75,2024-03-20,1,1,1.16,0.785714


In [43]:
save_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/risk_management/"
file_name = "calibrated_adjusted_index_predictions.csv"
file_path = save_path + file_name
final.to_csv(file_path)